In [1]:
import torch
import re
import random
import tqdm
import time
import numpy as np
import warnings
warnings.filterwarnings("ignore")

### Загрузим тексте и просто удалим лишние символы в тексте чтобы продемонстрировать работу GRU и LSTM ячеек:

In [2]:
# Ванильный препроцессинг:
def load_and_vanilla_preprocess(txt_path):
    with open(txt_path, encoding='utf-8') as txt_file:
        text = txt_file.read().lower()
    text = re.sub('[^a-z ]', ' ', text)
    text = re.sub('\s+', ' ', text)
    txt_file.close()
    return text

In [3]:
text = load_and_vanilla_preprocess('./data/nietzsche.txt')
text[:100]

'preface supposing that truth is a woman what then is there not ground for suspecting that all philos'

### Сформируем туже самую кодировку на уровне символов как делали до этого момента:

In [4]:
# Создаем индексы под символы
INDEX_TO_CHAR = sorted(list(set(text)))
# Кодировщик для символа в индекс:
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

In [5]:
print(CHAR_TO_INDEX)

{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


### Создадим кастомный датасет для работы:

In [6]:
MAX_LEN = 40 # берем кусочки текста максимум до 40 символов
STEP = 3
SENTENCES = []
NEXT_CHARS = []

# Проитерируемся по тексту и сформируем кусочки предложение (сэмплов)
for i in range(0, len(text) - MAX_LEN, STEP):
    SENTENCES.append(text[i: i + MAX_LEN])  # Формируем наши X
    NEXT_CHARS.append(text[i + MAX_LEN])    # Формируем Y

print("Количество предложений (сэмплов): ", len(SENTENCES))

Количество предложений (сэмплов):  193075


In [7]:
SENTENCES[:2]

['preface supposing that truth is a woman ',
 'face supposing that truth is a woman wha']

In [8]:
print("Векторизация...")

# Необходимо теперь нам это все превратить в тензоры для того чтобы подать в алгоритм рекуррентной нейронной сети
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)

# Пробегаемся по нашим кусочкам предложений и кодируем под сформированные символы
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

print(X.shape, Y.shape)
print(X[0:1], Y[0])

Векторизация...
torch.Size([193075, 40]) torch.Size([193075])
tensor([[16, 18,  5,  6,  1,  3,  5,  0, 19, 21, 16, 16, 15, 19,  9, 14,  7,  0,
         20,  8,  1, 20,  0, 20, 18, 21, 20,  8,  0,  9, 19,  0,  1,  0, 23, 15,
         13,  1, 14,  0]]) tensor(23)


### Теперь уже сформируем датасет, который будем уже передавать в нейронную сеть:

In [9]:
BATCH_SIZE = 512
dataset = torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)

In [10]:
# Строим класс RNN который будет принимать различную указанную вариацию рекуррентной ячейки - GRU / LSTM/ SimpleRNN
class RnnFlex(torch.nn.Module):
                        # тип     размер словаря  размер эмб       скрытые слои   классы
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        self.num_hiddens = num_hiddens
        self.embedding = torch.nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = torch.nn.Linear(num_hiddens, num_classes)

    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)  # приходят все выходы и последний выход (между LSTM  и GRU выход немного разный)
        predictions = self.output(state[0])
        return predictions

In [11]:
model = RnnFlex(torch.nn.GRU, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

In [12]:
model(X[0:1])

tensor([[-0.0895,  0.0958,  0.0262,  0.0139,  0.1866, -0.0922, -0.1018, -0.1026,
         -0.0376,  0.1924,  0.0942,  0.1496,  0.2639, -0.0695,  0.1569, -0.0483,
         -0.0315, -0.1166,  0.0543, -0.2714,  0.0421,  0.0222,  0.1019,  0.0220,
         -0.0871,  0.1240,  0.0551]], grad_fn=<AddmmBackward0>)

In [13]:
# Размерность возвращаемая GRU
embedding = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
rnn = torch.nn.GRU(28, 128, batch_first=True)
o, s = rnn(embedding(X[0:10]))
print("First Output shape: {}\nSecond output shape: {}".format(o.shape, s.shape))

First Output shape: torch.Size([10, 40, 128])
Second output shape: torch.Size([1, 10, 128])


In [14]:
# Размерность возвращаемая GRU
embedding = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
rnn = torch.nn.LSTM(28, 128)
o, s = rnn(embedding(X[0:1]))

print("First Output shape: {}\nSecond output shape: {}".format(o.shape, s.shape))

# print("First Output shape: {}\nSecond output shape: {} and {}".format(o.shape,  s[0].shape, s[1].shape))


AttributeError: 'tuple' object has no attribute 'shape'

#### Напишем 2 функции

1-ая отвечает за генерацию текста.

In [19]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    print("Softmaxed: {}".format(softmaxed))
    print("Softmaxed sum: {}".format(softmaxed.sum()))
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()


def generate_text():
    # Берем случайный индекс нашего текста
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index:start_index +MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred)[0]
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + ' <------ | ------> ' + generated[MAX_LEN: ])

In [20]:
generate_text()

Softmaxed: tensor([0.2387, 0.0494, 0.0029, 0.0059, 0.0058, 0.1193, 0.0028, 0.0052, 0.2522,
        0.0857, 0.0011, 0.0015, 0.0168, 0.0054, 0.0112, 0.0723, 0.0054, 0.0008,
        0.0170, 0.0248, 0.0266, 0.0148, 0.0038, 0.0021, 0.0008, 0.0271, 0.0006],
       grad_fn=<SoftmaxBackward0>)
Softmaxed sum: 0.9999999403953552
Softmaxed: tensor([2.1746e-01, 6.5274e-02, 1.1369e-03, 2.5414e-03, 4.0701e-03, 4.7272e-01,
        1.8442e-03, 2.8910e-03, 5.2706e-03, 1.0523e-01, 5.7465e-04, 1.0486e-03,
        1.0600e-02, 6.3467e-03, 4.9265e-03, 2.0271e-02, 5.7891e-03, 6.9149e-04,
        1.3502e-02, 1.5619e-02, 1.1209e-02, 8.6090e-03, 2.2783e-03, 1.6080e-03,
        5.0933e-04, 1.7544e-02, 4.3842e-04], grad_fn=<SoftmaxBackward0>)
Softmaxed sum: 0.9999999403953552
Softmaxed: tensor([5.1202e-01, 2.3460e-02, 2.3874e-03, 2.2088e-02, 2.9661e-02, 2.9653e-02,
        5.6716e-03, 1.1574e-02, 1.6474e-03, 1.8753e-02, 5.9918e-04, 1.2861e-03,
        2.2944e-02, 1.5676e-02, 4.6127e-02, 8.4931e-03, 7.4795e-03, 5.

In [21]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [22]:
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in data:
        optimizer.zero_grad()
        answers = model(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}   Time {:.3f}    Train Loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()

Epoch 0   Time 62.885    Train Loss: 1.963
Softmaxed: tensor([1.2259e-01, 1.3220e-03, 1.6164e-02, 2.5765e-02, 1.5682e-02, 2.3278e-03,
        5.8478e-03, 1.1352e-02, 2.6812e-03, 6.7338e-03, 3.0472e-04, 2.7508e-03,
        7.5778e-02, 1.4629e-02, 3.8778e-01, 2.3327e-03, 1.6488e-02, 3.4658e-04,
        7.0934e-02, 1.5389e-01, 4.2336e-02, 6.1115e-03, 6.4255e-03, 2.3435e-03,
        1.3139e-03, 5.5472e-03, 2.2531e-04], grad_fn=<SoftmaxBackward0>)
Softmaxed sum: 1.0
Softmaxed: tensor([6.2815e-02, 3.3710e-03, 1.7154e-03, 1.7865e-02, 8.1111e-01, 5.9134e-03,
        2.9292e-03, 6.9384e-03, 1.9094e-04, 6.9736e-03, 1.7013e-03, 5.5703e-03,
        4.4078e-03, 1.5250e-03, 2.1062e-03, 1.8107e-02, 4.1664e-04, 7.5076e-04,
        3.4838e-04, 1.0930e-02, 2.0007e-02, 6.8586e-04, 4.7228e-03, 2.0842e-03,
        4.2728e-04, 6.2601e-03, 1.2584e-04], grad_fn=<SoftmaxBackward0>)
Softmaxed sum: 0.9999998807907104
Softmaxed: tensor([9.4099e-01, 3.8085e-03, 1.5470e-04, 8.4100e-04, 8.1303e-04, 1.0843e-02,
     

KeyboardInterrupt: 